In [ ]:
# Import WRDS library
import wrds
import pandas as pd

## Establish connection to WRDS database

We initiate the database connection and assign it as a variable ```db```.<br>
Here, we need to key in our username and password.<br>
One can create a .pgpass file, but this is not required.

In [ ]:
# Establish live connection; requires user login (passwords will be masked)
db = wrds.Connection()

## Query libraries available on WRDS

In [ ]:
# Show list of available libraries
lib = db.list_libraries()

In [ ]:
print(f'WRDS contains {len(lib)} number of libraries.')

In [ ]:
# Show list of available libraries
lib = db.list_libraries()
om = [x for x in lib if 'option' in x]
om

## Query the tables within a given library

Here, we shall explore in a bit more depth what data tables are available in the ```optionm_all``` library.

In [ ]:
# Enquire existing available data tables
tables = db.list_tables(library='optionm_all')
print('There exists {} tables within the optionm_all library.'.format(len(tables)))
tables[0:5]

# Inspect given data tables

Here, we are interested in the following library.table:
1. cboe_all.cboe: to obtain VIX index data
2. optionm_all.secprd: to obtain S&P500 index data
3. optionm_all.opvold: to obtain S&P500 option data


In [ ]:
db.describe_table(library = 'cboe_all', table = 'cboe' )

The ```cboe_all.cboe``` table contains the OHLC data for multiple volatility indices, including VIX, VNasdaq, VDJIA.

In [ ]:
db.describe_table(library='optionm_all', table='secprd')

for the ```optionm_all.secprd``` table, the key information are: ```secid```, ```date```, ```low```, ```high```, ```close```, ```open```, ```volume```.

In [ ]:
db.describe_table(library = 'optionm_all', table = 'opvold')

## Data extraction using SQL query on WRDS

The ```optionm_all.opvold``` table consists of the option volume and open interest, split by ```cp_flag```.

In [ ]:
# Get VIX
# library = 'cboe_all', table = 'cboe' 
vix_fn = "../SQL/get_vix.sql"

# Get S&P 500 Index
# S&P 500 index price:  library='optionm_all', table='secprd'
spx_fn = "../SQL/get_spx.sql"

# Get S&P 500 Call, Put options volume and flag
# volume: library = optionm_all, table = opvold
opvol_fn = "../SQL/get_option_volume.sql"



In [ ]:
fd = open(spx_fn, 'r')
sqlFile = fd.read()
fd.close()

print(sqlFile)

In [ ]:
def read_sql_script(fname, db, date_cols = ['trading_date']):
    fd = open(fname, 'r')
    sqlFile = fd.read()
    fd.close()

    df = db.raw_sql(sqlFile, date_cols=date_cols)

    return df

In [ ]:
df_vix = read_sql_script(vix_fn, db)
print('VIX data extracted')

df_vix.head()

In [ ]:
df_spx = read_sql_script(spx_fn, db)
print('SPX data extracted')

df_spx.head()

In [ ]:
df_opvol = read_sql_script(opvol_fn, db)
print('option volume data extracted')

df_opvol.head()